In [1]:
import os, csv, json
from xml.etree import ElementTree as ET

In [2]:
def search_table(row, table, has_id):
    if has_id: #if the row in the table has a given id, check only if the id is already present
        if table.get(row[0]) != None:
            return int(row[0])
        else:
            return -1

    #otherwise search all the elements, if the same row is in the table, then we do not insert
    for key, value in table.items():
        if row == value:
            return int(key)
    
    return -1

In [3]:
def insertRow(row, table, has_id):
    if not table: #check if is void
        # if not void, use the row with id if it's given, otherwise create a counter
        if not has_id:
            table[1] = row
        else:
            table[row[0]] = row[1:]
        return 1
    
    # not void
    found = search_table(row, table, has_id)
    if found >= 0: #if element already in the table, return its id
        return found
    # use the id in the row if present, otherwise increment the counter (last insert + 1)
    if not has_id:
        new_id = list(table.keys())[-1] + 1
        table[new_id] = row
        return new_id
    
    table[row[0]] = row[1:]
    return row[0]

In [4]:
#come detto da Vale
def insertGeo(row, table):
    id = str(row[0])+str(row[1])
    if not table: # check if is void
        table[id] = row
        #unique_locations.add(tuple(row))
        return id
    
    # not void
    if table.get(id) != None: #if element already in the table, return its id
        return id
    # use the id in the row if present, otherwise increment the counter (last insert + 1) 
    table[id] = row
    return id

In [6]:
def split_Police(row, custody, gun, geography, partecipant, incident, date, age_dict, type_dict, status_dict):
    # use different list to compute the rows of the different tables
    cus_row = []
    gun_row = []
    geo_row = []
    part_row = []
    date_row = []
    

    #split different values based on the tables
    for key, value in row.items():
        if key == 'custody_id':
            cus_row.append(value)
        elif key == 'participant_age_group' or key == 'participant_gender' or key == 'participant_status' or key == 'participant_type':
            part_row.append(value)
        elif key == 'latitude' or key == 'longitude':
            geo_row.append(float(value))
        elif key == 'gun_stolen' or key == 'gun_type':
            gun_row.append(value)
        elif key == 'incident_id':
            incident_id  = int(value)
        elif key == 'date_fk':
            date_row.append(int(value))
        else:
            print('key not recognized')


    crime_gravity = age_dict[part_row[0]] * type_dict[part_row[3]] * status_dict[part_row[2]]
    id_part = insertRow(part_row, partecipant, False)
    id_gun = insertRow(gun_row, gun, False)
    incident.add(incident_id)
    id_geo = insertGeo(geo_row, geography)
    id_date = insertRow(date_row, date, True)
    cus_row = cus_row + [id_part] + [id_gun] + [id_geo] + [id_date] + [incident_id] + [crime_gravity]
    #cus_row = cus_row + [id_part] + [id_gun] + [id_date] + [incident_id] + [crime_gravity]
    custody.append(cus_row)



In [7]:
#create list version of the tables
custody = list()
gun = dict()
geography = dict()
partecipant = dict()
incident = set() #since incident has only the id (integer) use a set to speed up
date = dict()
i = 0

#retrieve transformation to compute the crime score
with open('dict_partecipant_age.json') as d1:
    age_fun = json.load(d1)
with open('dict_partecipant_type.json') as d2:
    type_fun = json.load(d2)
with open('dict_partecipant_status.json') as d3:
    status_fun = json.load(d3)

#read line by line the file
with open('Police.csv') as f1:
    police = csv.DictReader(f1)
    for row in police:
        i += 1
        #print(row)
        split_Police(row, custody, gun, geography, partecipant, incident, date, age_fun, type_fun, status_fun)
        

In [52]:
#open xml file
tree = ET.parse(os.getcwd()+'/lds_project_2023/dates.xml')
root = tree.getroot()

#create dictionary where the keys the primary keys of the table, the values are the dates
date_info = {}
for row in root:
    date_info[row.findtext('date_pk')] = row.findtext('date')

#update the date table
for key, value in date_info.items():
    date[int(key)] = value

170 928